In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(151)

# 演習2

## データのロード

In [ ]:
train_df = pd.read_csv("data/train_enshu2.csv", index_col=0)

In [ ]:
test_df = pd.read_csv("data/test_enshu2.csv", index_col=0)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## データの正規化

In [ ]:
from sklearn.preprocessing import MinMaxScaler

演習1と同様、移動平均線からの乖離を学習させましょう。

In [ ]:
diff_ma_train = train_df[["count"]] - train_df[["count"]].rolling(7).mean()
diff_ma_test = test_df[["count"]] - test_df[["count"]].rolling(7).mean()

In [ ]:
diff_ma_train.dropna(inplace=True)
diff_ma_test.dropna(inplace=True)

In [ ]:
diff_ma_train.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
train = scaler.fit_transform(diff_ma_train)
test = scaler.transform(diff_ma_test)

データを短い単位に分割(並列処理のため)

In [ ]:
train.shape

In [ ]:
xs = []
i = 0
while i + 21 <= train.shape[0]:
    xs.append(train[i:i + 21])
    i += 14
xs = np.array(xs)

In [ ]:
xs.shape

予測値を作成

In [ ]:
ys = []
i = 1
while i + 21 <= train.shape[0]:
    ys.append(train[i:i + 21])
    i += 14
ys = np.array(ys)

In [ ]:
ys.shape

## 学習済みモデルをロード

In [ ]:
import keras

In [ ]:
model = # TODO 学習済みモデルをロードしてください(keras.models.load_modelを使います。)

In [ ]:
%%time
history = # TODO 学習しましょう(fitを使います。)

## テスト

In [ ]:
test.shape

In [ ]:
pred_train = model.predict(train[np.newaxis, :, :])
pred = model.predict(test[np.newaxis, :, :])

In [ ]:
pred_train.shape, pred.shape

In [ ]:
t_train = train.ravel()[1:]
p_train = pred_train.ravel()[:-1]
t = test.ravel()[1:]
p = pred.ravel()[:-1]

In [ ]:
t_train.shape, p_train.shape, t.shape, p.shape

学習データに対するFitを見てみましょう

In [ ]:
pd.DataFrame(dict(t=t_train, p=p_train)).plot()

テストデータに対するFitを見てみましょう。

In [ ]:
pd.DataFrame(dict(t=t, p=p)).plot()

## 予測値に逆変換します

In [ ]:
pred.shape

In [ ]:
transformed = test_df[["count"]].rolling(7).mean().values[6:] + scaler.inverse_transform(pred[0])

In [ ]:
test_df["count"].values.shape

In [ ]:
pd.DataFrame(dict(t=test_df["count"].values[7:], p=transformed.ravel()[:-1])).plot()

## 試しにFineTuning前のモデルで結果を見てみましょう

In [ ]:
basemodel = keras.models.load_model("data/pretrain_enshu2.h5")

In [ ]:
pred_train = basemodel.predict(train[np.newaxis, :, :])
pd.DataFrame(dict(t=train.ravel()[1:], p=pred_train.ravel()[:-1])).plot()

In [ ]:
pred = basemodel.predict(test[np.newaxis, :, :])
pd.DataFrame(dict(t=t, p=pred.ravel()[:-1])).plot()

## (参考) Fine Tuningしなかったらどうなるのか

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, Activation, Dropout

In [ ]:
no_ft = Sequential()
no_ft.add(LSTM(input_dim=1, output_dim=50, return_sequences=True))
no_ft.add(Dropout(0.5))
no_ft.add(Activation("tanh"))
no_ft.add(LSTM(output_dim=10, return_sequences=True))
no_ft.add(Dropout(0.5))
no_ft.add(Activation("tanh"))
no_ft.add(TimeDistributed(Dense(output_dim=5)))
no_ft.add(Dropout(0.5))
no_ft.add(Activation("relu"))
no_ft.add(TimeDistributed(Dense(output_dim=1, activation="tanh")))
no_ft.compile(loss="mean_squared_error", optimizer="adam")

In [ ]:
%%time
history_no_ft = no_ft.fit(xs[:-1], ys, nb_epoch=100, validation_split=0.1, verbose=0)

In [ ]:
pred_train = no_ft.predict(train[np.newaxis, :, :])
pd.DataFrame(dict(t=train.ravel()[1:], p=pred_train.ravel()[:-1])).plot()

In [ ]:
pred = no_ft.predict(test[np.newaxis, :, :])
pd.DataFrame(dict(t=t, p=pred.ravel()[:-1])).plot()

学習が収束する様子を比較しましょう。

In [ ]:
history.history.keys()

In [ ]:
history_no_ft.history.keys()

In [ ]:
h = {}
h['loss_ft'] = history.history['loss']
h['val_loss_ft'] = history.history['val_loss']
h['loss_no_ft'] = history_no_ft.history['loss']
h['val_loss_no_ft'] = history_no_ft.history['val_loss']
pd.DataFrame(h).plot()